In [173]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import datetime

pd.options.display.max_columns = 8
pd.options.display.max_rows = 50

import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

SOURCE_PATH = '../data/source/'
SUBMISSION_PATH = '../data/submission/'

In [174]:
def encode_cardinality(data, feature):
    data[feature].fillna('nan', inplace=1)
    feature_dict = dict(data[feature].value_counts(dropna=False))
    data[feature] = data[feature].map(lambda x: feature_dict[x])
    return data

def parse_date(date, target='year'):
    if hasattr(date, target):
        return getattr(date, target)
    elif type(date) is str:
        if len(date.split(' ')[0].split('/')) == 3:
            if target == 'year':
                return int(date.split(' ')[0].split('/')[2])
            elif target == 'month':
                return int(date.split(' ')[0].split('/')[0])
        else:
            return np.nan
        
def one_hot_encode(data, features):
    le = LabelEncoder()
    onehot = OneHotEncoder(sparse=False)
    for feature in features:
        data[feature] = le.fit_transform(data[feature])
        labels = le.classes_
        hot = onehot.fit_transform(np.array(data[[feature]]))
        temp = pd.DataFrame(hot, columns=[feature + '_' + str(label) for label in labels])
        if data[feature].dtype == object:
            data.drop(feature, axis=1, inplace=True)
        data = data.merge(temp, left_index=True, right_index=True)
        data.drop(feature, axis=1, inplace=True)
    return data

## Data import

In [175]:
train = pd.read_excel(os.path.join(SOURCE_PATH, 'train.xlsx'), sheetname=1, encoding='utf-8')
test = pd.read_excel(os.path.join(SOURCE_PATH, 'test.xlsx'), sheetname=1, encoding='utf-8')

In [176]:
train['is_train'] = 1
test['is_train'] = 0
data = pd.concat([train, test])

## Preprocessing

### Target feature

In [177]:
data['target'] = data['OPPORTUNITY STAGENAME'].map(lambda x: int(x == 'Closed Won'))
if data.filter(like='OPP').shape[1] > 0:
    data.drop(train.filter(like='OPP').columns, axis=1, inplace=True)

### Useless features

In [178]:
drop_features = ['EMAIL', 'CONTACT QUALIFICATION', 'CONTACT.QUALIFIEDDATE__C', 'COMAPNYNAME']
for feature in drop_features:
    if feature in data.columns:
        data.drop(feature, axis=1, inplace=True)
    else: print "ERROR, no such feature", feature

### Add nan features

In [179]:
nan_features = ['zip_set', 'State', 'PRODUCTLIST__C']
for feature in nan_features:
    if feature in data.columns:
        data[feature+'_isnan'] = data[feature].isnull().astype(int)
    else: print "ERROR, no such feature", feature

### Cardinality encoding

In [180]:
card_features = ['zip_set', 'State', 'PRODUCTLIST__C', 'CAMPAIGN.CAMPAIGN_CODE__C', 'CAMPAIGN.NAME', 
                 'CAMPAIGN.SUB_TYPE__C', 'ACTIVITY FILENAME__C', 'ACTIVITY_NAME']
for feature in card_features:
    if feature in data.columns:
        data = encode_cardinality(data, feature)
    else: print "ERROR, no such feature", feature

### One-Hot binarization

In [181]:
onehot_features = ['RECORD-TYPE', 'CAMPAIGN.TYPE', 'CAMPAIGN_DEPARTMENT']
data = one_hot_encode(data, onehot_features)

### Special features processing

In [182]:
if 'RECORD_CREATEDDATE' in data.columns:
    data['rec_cd_month'] = data.RECORD_CREATEDDATE.map(lambda x: x.month)
    data.rec_cd_month.fillna(-1, inplace=True)
    data.drop('RECORD_CREATEDDATE', axis=1, inplace=True)

In [183]:
if 'LEAD_CONTACT_CREATEDDATE' in data.columns:
    data['lc_cd_month'] = data.LEAD_CONTACT_CREATEDDATE.map(lambda x: parse_date(x, target='month'))
    data['lc_cd_year'] = data.LEAD_CONTACT_CREATEDDATE.map(lambda x: parse_date(x, target='year'))
    data.drop('LEAD_CONTACT_CREATEDDATE', axis=1, inplace=True)
data.lc_cd_month.fillna(-1, inplace=True)
data.lc_cd_year.fillna(-1, inplace=True)

In [184]:
if 'CREATEDBYID' in data.columns:
    data['cr_byid_7MY1AAM'] = (data.CREATEDBYID == '005600000017MY1AAM').astype(int)
    data['cr_byid_VxfoAAC'] = (data.CREATEDBYID == '00560000001VxfoAAC').astype(int)
    data['cr_byid_other'] = ((data.CREATEDBYID != '00560000001VxfoAAC') & (data.CREATEDBYID != '005600000017MY1AAM')).astype(int)
    data.drop('CREATEDBYID', axis=1, inplace=True)

In [185]:
if 'COUNTRY' in data.columns:
    data['is_Germany'] = (data.COUNTRY == 'Germany').astype(int)
    data['is_USA'] = (data.COUNTRY == 'United States').astype(int)
    data['country_isnan'] = data.COUNTRY.isnull().astype(int)
    data['country_other'] = ((data.COUNTRY.notnull()) & (data.COUNTRY != 'Germany') & (data.COUNTRY != 'United States')).astype(int)
    data.drop('COUNTRY', axis=1, inplace=True)

In [186]:
if 'ACTIVITY_TYPE__C' in data.columns:
    top = ['Registration', 'License Generation', 'Download', 'Download resource'] 
    data['act_na'] = data.ACTIVITY_TYPE__C.isnull().astype(int)
    data['act_registration'] = (data.ACTIVITY_TYPE__C == 'Registration').astype(int)
    data['act_license'] = (data.ACTIVITY_TYPE__C == 'License Generation').astype(int)
    data['act_download'] = (data.ACTIVITY_TYPE__C == 'Download').astype(int)
    data['act_download_res'] = (data.ACTIVITY_TYPE__C == 'Download resource').astype(int)
    data['act_other'] = ((data.ACTIVITY_TYPE__C.notnull()) & 
                         (data.ACTIVITY_TYPE__C.map(lambda x: x not in top))
                        ).astype(int)
    data.drop('ACTIVITY_TYPE__C', axis=1, inplace=True)

## Grouping by domain

In [187]:
main_data = data.groupby('DOMAIN').mean()
data['e_count'] = np.ones((data['DOMAIN'].shape))

## Email_counts feature

In [ ]:
e_count = data[['DOMAIN', 'e_count']].groupby('DOMAIN').count()
main_data = main_data.merge(e_count, left_index=True, right_index=True)

## Train test split

In [188]:
train = main_data[main_data.is_train == 1]
test = main_data[main_data.is_train == 0]
print train.shape, test.shape

x_train = np.array(train.drop(['target', 'is_train'], axis=1))
y_train = np.array(train.target)
x_test = np.array(test.drop(['target', 'is_train'], axis=1))
y_test = np.array(test.target)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(22718, 53) (22357, 53)
(22718, 51) (22357, 51) (22718,) (22357,)


## Model

In [191]:
import xgboost as xgb

model = xgb.XGBClassifier(max_depth=4, learning_rate=0.02, n_estimators=600,
                          silent=1, objective="binary:logistic", min_child_weight=5, seed=42)
model.fit(x_train, y_train)
y_pred = model.predict_proba(x_test)[:, 1]
submisson = pd.DataFrame({'is_Closed_Won_prob':y_pred}, index=test.index)
submisson.to_csv(os.path.join(SUBMISSION_PATH, 'veeam_results2.csv'))